# 📝 2024년 데이터마이닝 중간고사 기출문제 - 완벽 정리본
## ⭐ 시험장에서 바로 이해할 수 있도록 주석 완비!

# 문제 1. 로지스틱 회귀분석 (Phishing 분류) - 40점

## 🎯 목표: 웹사이트가 피싱 사이트인지 분류
- **데이터**: web-page-phishing.csv
- **목표변수**: phishing (1: 피싱, 0: 정상)
- **모델**: 로지스틱 회귀
- **핵심 평가지표**: F1 Score, ROC AUC

## 라이브러리 불러오기

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

# 🔹 분류 모델 라이브러리 (문제 1, 2용)
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression  # ← 로지스틱 회귀
from sklearn.tree import DecisionTreeClassifier, plot_tree  # ← 의사결정트리
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# 🔹 회귀 모델 라이브러리 (문제 3용)
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

# dmba 라이브러리 (선택사항)
try:
    from dmba import classificationSummary, regressionSummary
    HAS_DMBA = True
    print("✅ dmba 라이브러리 사용 가능")
except:
    HAS_DMBA = False
    print("⚠️ dmba 없음 - sklearn metrics만 사용")

## 문제 1-(1) 데이터 전처리

### 📌 요구사항
- 입력변수: phishing 제외한 모든 변수
- 데이터 전처리 수행

In [ ]:
# ========================================
# 1-(1) 데이터 로드 및 탐색
# ========================================

# 데이터 로드
df = pd.read_csv('web-page-phishing.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 데이터 정보")
print("="*60)
df.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플 (처음 5개)")
print("="*60)
print(df.head())

In [ ]:
# ========================================
# 변수 타입 자동 분류
# ========================================

print("="*60)
print("🔍 변수 타입 분류")
print("="*60)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n범주형 변수 ({len(categorical_cols)}개): {categorical_cols}")
print(f"수치형 변수 ({len(numerical_cols)}개): {numerical_cols}")

# 💡 이 데이터는 모두 수치형일 가능성 높음 (URL 특징들)

In [ ]:
# ========================================
# 독립변수 / 종속변수 분리
# ========================================

# 🎯 목표변수 설정
outcome = 'phishing'  # ← 문제에서 지정

# 🔹 독립변수 = 목표변수 제외한 모든 변수
predictors = [col for col in df.columns if col != outcome]

X = df[predictors]
y = df[outcome]

print("="*60)
print("📊 변수 분리 완료")
print("="*60)
print(f"독립변수 개수: {len(predictors)}개")
print(f"데이터 크기: {X.shape}")
print(f"\n목표변수 분포:")
print(y.value_counts())
print(f"\n목표변수 비율:")
print(y.value_counts(normalize=True))

# 💡 이진 분류 문제 확인!

In [ ]:
# ========================================
# 범주형 변수 원-핫 인코딩 (필요시)
# ========================================

# 이 데이터는 모두 수치형이라 인코딩 불필요할 가능성 높음
# 하지만 범주형이 있다면 아래 코드 실행

if len(categorical_cols) > 0:
    print(f"\n범주형 변수 인코딩 중: {categorical_cols}")
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    print(f"인코딩 후 변수 개수: {X.shape[1]}개")
else:
    print("\n✅ 범주형 변수 없음 - 인코딩 불필요")

print(f"\n최종 독립변수 개수: {X.shape[1]}개")

## 문제 1-(2) 데이터 분리

### 📌 요구사항
- 무작위 분리
- **테스트 20%**, 학습 80%
- random_state 설정 (재현성)

In [ ]:
# ========================================
# 1-(2) 학습/테스트 데이터 분할
# ========================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # ← ⭐ 문제에서 지정: 테스트 20%
    random_state=42,    # ← 재현성 (아무 숫자나 가능, 1도 괜찮음)
    stratify=y          # ← 클래스 비율 유지 (권장)
)

print("="*60)
print("✂️ 데이터 분리 완료")
print("="*60)
print(f"학습 데이터: {X_train.shape} (80%)")
print(f"테스트 데이터: {X_test.shape} (20%)")
print(f"\n학습 데이터 클래스 분포:")
print(y_train.value_counts())
print(f"\n테스트 데이터 클래스 분포:")
print(y_test.value_counts())

## 문제 1-(3) 로지스틱 회귀 학습 및 F1 Score 측정

### 📌 요구사항
- 로지스틱 회귀 모델 학습
- 테스트 데이터로 예측
- **F1 Score로 평가** ← ⭐ 핵심!

In [ ]:
# ========================================
# 1-(3) 로지스틱 회귀 모델 학습
# ========================================

print("="*60)
print("📊 로지스틱 회귀 모델 (기본)")
print("="*60)

# 모델 생성 및 학습
logreg = LogisticRegression(
    max_iter=1000,      # ← 수렴 보장
    random_state=42     # ← 재현성
)
logreg.fit(X_train, y_train)

print("✅ 모델 학습 완료!")

In [ ]:
# ========================================
# 예측 수행
# ========================================

# 학습 데이터 예측
y_pred_train = logreg.predict(X_train)

# 테스트 데이터 예측 (⭐ 문제에서 요구)
y_pred_test = logreg.predict(X_test)

print("✅ 예측 완료!")

In [ ]:
# ========================================
# F1 Score 측정 (⭐ 문제에서 요구!)
# ========================================

# 학습 데이터 F1 Score
f1_train = f1_score(y_train, y_pred_train)

# 테스트 데이터 F1 Score (⭐ 핵심!)
f1_test = f1_score(y_test, y_pred_test)

print("\n" + "="*60)
print("📊 문제 1-(3) 결과: F1 Score")
print("="*60)
print(f"학습 데이터 F1 Score:  {f1_train:.4f}")
print(f"테스트 데이터 F1 Score: {f1_test:.4f}  ← ⭐ 답안!")
print("="*60)

# 추가 지표들 (참고용)
print(f"\n[추가 성과 지표 - 테스트 데이터]")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_test):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_test):.4f}")

## 문제 1-(4) 5겹 교차검증 (F1 Score)

### 📌 요구사항
- **5겹 교차검증**
- **F1 Score로 평가**
- **solver='newton-cholesky'** ← ⭐ 문제에서 지정!
- 평균치 출력

In [ ]:
# ========================================
# 1-(4) 5겹 교차검증 (F1 Score)
# ========================================

print("="*60)
print("🔄 5겹 교차검증 (solver='newton-cholesky')")
print("="*60)

# 모델 생성 (⭐ solver 지정 필수!)
logreg_cv = LogisticRegression(
    solver='newton-cholesky',  # ← ⭐ 문제에서 지정!
    max_iter=1000,             # ← 수렴 보장
    random_state=42
)

# 5겹 교차검증 수행
cv_scores = cross_val_score(
    logreg_cv, 
    X_train,        # ← 학습 데이터만 사용
    y_train, 
    cv=5,           # ← ⭐ 5겹
    scoring='f1',   # ← ⭐ F1 Score
    n_jobs=-1
)

print("\n" + "="*60)
print("📊 문제 1-(4) 결과: 5겹 교차검증")
print("="*60)
print(f"5-Fold CV F1 Score: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
print(f"\n각 Fold 점수: {cv_scores}")
print(f"\n평균: {cv_scores.mean():.4f}  ← ⭐ 답안!")
print("="*60)

## 문제 1-(5) 하이퍼파라미터 튜닝 (C 튜닝)

### 📌 요구사항
- **파라미터**: C (정규화 강도의 역수)
- **평가 지표**: ROC AUC ← ⭐ F1에서 변경!
- **교차검증**: 10겹 ← ⭐ 5겹에서 변경!
- **solver='newton-cholesky'**
- 최적 C 값 찾기

In [ ]:
# ========================================
# 1-(5) C 하이퍼파라미터 튜닝
# ========================================

print("="*60)
print("🔧 하이퍼파라미터 튜닝 (C)")
print("="*60)

# C 값 후보 정의
# C의 default = 1.0
# C가 작을수록 강한 규제, 클수록 약한 규제
param_grid = {
    'C': np.logspace(-3, 3, 20)  # 0.001 ~ 1000 사이 20개 값
}

# GridSearchCV 설정
grid_search = GridSearchCV(
    LogisticRegression(
        solver='newton-cholesky',  # ← ⭐ 문제에서 지정!
        max_iter=1000,
        random_state=42
    ),
    param_grid,
    cv=10,              # ← ⭐ 10겹 교차검증
    scoring='roc_auc',  # ← ⭐ ROC AUC로 평가
    n_jobs=-1,
    verbose=1
)

print("\n⏳ GridSearchCV 진행 중...")
grid_search.fit(X_train, y_train)

print("\n" + "="*60)
print("📊 문제 1-(5) 결과: 최적 C 값")
print("="*60)
print(f"최적 C 값: {grid_search.best_params_['C']:.6f}  ← ⭐ 답안!")
print(f"최고 CV ROC AUC: {grid_search.best_score_:.4f}")
print("="*60)

In [ ]:
# ========================================
# 최적 모델로 테스트 데이터 평가
# ========================================

best_logreg = grid_search.best_estimator_

# 테스트 데이터 예측
y_pred_test_best = best_logreg.predict(X_test)
y_pred_proba_test = best_logreg.predict_proba(X_test)[:, 1]

# 성과 측정
f1_test_best = f1_score(y_test, y_pred_test_best)
roc_auc_test = roc_auc_score(y_test, y_pred_proba_test)

print("\n" + "="*60)
print("📊 최적 모델 테스트 성과")
print("="*60)
print(f"F1 Score:  {f1_test_best:.4f}")
print(f"ROC AUC:   {roc_auc_test:.4f}")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_test_best):.4f}")
print("="*60)

---

# 문제 2. 의사결정트리 (Phishing 분류) - 20점

## 🎯 목표: 같은 데이터로 의사결정트리 모델 만들기
- **데이터**: 문제 1과 동일 (web-page-phishing.csv)
- **모델**: 의사결정트리
- **핵심**: max_depth 튜닝

## 문제 2-(1) 최대 나무 모델 (Full Tree)

### 📌 요구사항
- 문제 1의 전처리된 학습집합 사용
- 최대 나무 (제약 없음)
- 테스트집합으로 F1 Score 측정
- default 불순도 지표 사용

In [ ]:
# ========================================
# 2-(1) 최대 의사결정트리 모델
# ========================================

print("="*60)
print("🌳 의사결정트리 - 최대 나무 (Full Tree)")
print("="*60)

# 모델 생성 (제약 없음)
tree_full = DecisionTreeClassifier(
    random_state=42  # ← 재현성
    # criterion='gini'  ← default (생략 가능)
    # max_depth=None    ← default (생략 가능)
)

# 학습
tree_full.fit(X_train, y_train)

print("✅ 모델 학습 완료!")
print(f"\n[나무 구조 정보]")
print(f"최대 깊이: {tree_full.get_depth()}")
print(f"리프 노드 수: {tree_full.get_n_leaves()}")
print(f"총 노드 수: {tree_full.tree_.node_count}")

In [ ]:
# ========================================
# 테스트 데이터 평가 (F1 Score)
# ========================================

# 예측
y_pred_test_tree = tree_full.predict(X_test)

# F1 Score 계산
f1_test_tree_full = f1_score(y_test, y_pred_test_tree)

print("\n" + "="*60)
print("📊 문제 2-(1) 결과: 최대 나무 F1 Score")
print("="*60)
print(f"테스트 데이터 F1 Score: {f1_test_tree_full:.4f}  ← ⭐ 답안!")
print("="*60)

# 추가 지표
print(f"\n[추가 성과 지표]")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_test_tree):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_test_tree):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_test_tree):.4f}")

## 문제 2-(2) max_depth 튜닝

### 📌 요구사항
- max_depth를 2~20까지 변화
- 각 depth마다 F1 Score 측정
- 학습/테스트 데이터로 성과 비교
- 가장 좋은 max_depth 찾기

In [ ]:
# ========================================
# 2-(2) max_depth 튜닝
# ========================================

print("="*60)
print("🔧 max_depth 튜닝 (2~20)")
print("="*60)

depths = range(2, 21)  # 2부터 20까지
train_f1_scores = []
test_f1_scores = []

for depth in depths:
    # 모델 생성
    tree = DecisionTreeClassifier(
        max_depth=depth,
        random_state=42
    )
    
    # 학습
    tree.fit(X_train, y_train)
    
    # 학습 데이터 F1 Score
    train_pred = tree.predict(X_train)
    train_f1 = f1_score(y_train, train_pred)
    train_f1_scores.append(train_f1)
    
    # 테스트 데이터 F1 Score
    test_pred = tree.predict(X_test)
    test_f1 = f1_score(y_test, test_pred)
    test_f1_scores.append(test_f1)

# 최적 depth 찾기
best_depth_idx = np.argmax(test_f1_scores)
best_depth = list(depths)[best_depth_idx]
best_f1 = test_f1_scores[best_depth_idx]

print("\n" + "="*60)
print("📊 문제 2-(2) 결과: 최적 max_depth")
print("="*60)
print(f"최적 max_depth: {best_depth}  ← ⭐ 답안!")
print(f"테스트 F1 Score: {best_f1:.4f}")
print("="*60)

In [ ]:
# ========================================
# max_depth별 성과 시각화
# ========================================

plt.figure(figsize=(12, 6))
plt.plot(depths, train_f1_scores, 'o-', label='Train F1', linewidth=2, markersize=6)
plt.plot(depths, test_f1_scores, 's-', label='Test F1', linewidth=2, markersize=6)
plt.axvline(best_depth, color='r', linestyle='--', linewidth=2, 
            label=f'Best depth={best_depth}')
plt.xlabel('max_depth', fontsize=12, fontweight='bold')
plt.ylabel('F1 Score', fontsize=12, fontweight='bold')
plt.title('F1 Score vs max_depth', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 과적합 여부 확인:")
print(f"학습 F1: {train_f1_scores[best_depth_idx]:.4f}")
print(f"테스트 F1: {test_f1_scores[best_depth_idx]:.4f}")
print(f"차이: {train_f1_scores[best_depth_idx] - test_f1_scores[best_depth_idx]:.4f}")

## 문제 2-(3) 최적 모델의 변수 중요도

### 📌 요구사항
- 2-(2)에서 찾은 최적 depth 모델 사용
- 입력 변수 중요도 파악

In [ ]:
# ========================================
# 2-(3) 변수 중요도 분석
# ========================================

# 최적 depth로 모델 재학습
tree_best = DecisionTreeClassifier(
    max_depth=best_depth,
    random_state=42
)
tree_best.fit(X_train, y_train)

# 변수 중요도 추출
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': tree_best.feature_importances_
}).sort_values('importance', ascending=False)

print("="*60)
print("📊 문제 2-(3) 결과: 변수 중요도")
print("="*60)
print("\n[Top 10 중요 변수]")
print(importance_df.head(10).to_string(index=False))
print("="*60)

In [ ]:
# 시각화
plt.figure(figsize=(10, 8))
top_15 = importance_df.head(15)
plt.barh(range(len(top_15)), top_15['importance'])
plt.yticks(range(len(top_15)), top_15['feature'])
plt.xlabel('Importance', fontsize=12, fontweight='bold')
plt.title('Top 15 Feature Importances', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

---

# 문제 3. 회귀분석 (Insurance 청구비용 예측) - 35점

## 🎯 목표: 보험 청구비용 예측
- **데이터**: insurance.csv
- **목표변수**: charges (청구비용)
- **모델**: Linear Regression, Lasso
- **핵심 평가지표**: MAPE, MSE

## 문제 3-(1) 데이터 전처리

### 📌 요구사항
- 종속변수: charges
- 독립변수: 나머지 모든 변수
- **범주형 변수**: 원-핫 인코딩
- **수치형 변수**: 표준화

In [ ]:
# ========================================
# 3-(1) 데이터 로드 및 탐색
# ========================================

df_ins = pd.read_csv('insurance.csv')  # ← ⭐ 파일명 확인!

print("="*60)
print("📊 Insurance 데이터 정보")
print("="*60)
df_ins.info()

print("\n" + "="*60)
print("🔍 결측치 확인")
print("="*60)
print(df_ins.isnull().sum())

print("\n" + "="*60)
print("👀 데이터 샘플")
print("="*60)
print(df_ins.head())

In [ ]:
# ========================================
# 변수 타입 분류
# ========================================

print("="*60)
print("🔍 변수 타입 분류")
print("="*60)

categorical_cols_ins = df_ins.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols_ins = df_ins.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\n범주형 변수: {categorical_cols_ins}")
print(f"수치형 변수: {numerical_cols_ins}")

In [ ]:
# ========================================
# 독립변수/종속변수 분리
# ========================================

outcome_ins = 'charges'  # ← 목표변수
predictors_ins = [col for col in df_ins.columns if col != outcome_ins]

X_ins = df_ins[predictors_ins]
y_ins = df_ins[outcome_ins]

print("="*60)
print("📊 변수 분리")
print("="*60)
print(f"독립변수: {predictors_ins}")
print(f"종속변수: {outcome_ins}")
print(f"\n종속변수 기본 통계:")
print(y_ins.describe())

In [ ]:
# ========================================
# 범주형 변수 원-핫 인코딩
# ========================================

# 범주형 변수만 인코딩
categorical_in_X = [col for col in categorical_cols_ins if col in X_ins.columns]

if len(categorical_in_X) > 0:
    print(f"\n원-핫 인코딩 대상: {categorical_in_X}")
    X_ins = pd.get_dummies(X_ins, columns=categorical_in_X, drop_first=True)
    print(f"인코딩 후 변수 개수: {X_ins.shape[1]}개")

print(f"\n최종 독립변수 목록: {X_ins.columns.tolist()}")

In [ ]:
# ========================================
# 수치형 변수 표준화
# ========================================

# 수치형 변수 찾기 (원-핫 인코딩 후 기준)
numerical_in_X = X_ins.select_dtypes(include=['int64', 'float64']).columns.tolist()

if len(numerical_in_X) > 0:
    scaler_ins = StandardScaler()
    X_ins[numerical_in_X] = scaler_ins.fit_transform(X_ins[numerical_in_X])
    print(f"\n✅ 표준화 완료: {numerical_in_X}")

print(f"\n최종 데이터 크기: {X_ins.shape}")

## 문제 3-(2) 회귀분석 모형 학습

### 📌 요구사항
- 전체 데이터를 **20% 테스트**, 80% 학습으로 분리
- 학습집합으로 회귀분석 모형 학습
- 청구비용에 유의미한 영향을 미치는 변수 파악

In [ ]:
# ========================================
# 3-(2) 데이터 분리
# ========================================

X_train_ins, X_test_ins, y_train_ins, y_test_ins = train_test_split(
    X_ins, y_ins,
    test_size=0.2,     # ← ⭐ 테스트 20%
    random_state=42
)

print("="*60)
print("✂️ 데이터 분리")
print("="*60)
print(f"학습 데이터: {X_train_ins.shape} (80%)")
print(f"테스트 데이터: {X_test_ins.shape} (20%)")

In [ ]:
# ========================================
# 회귀분석 모형 학습
# ========================================

print("\n" + "="*60)
print("📊 Linear Regression 학습")
print("="*60)

lr_ins = LinearRegression()
lr_ins.fit(X_train_ins, y_train_ins)

print("✅ 모델 학습 완료!")

In [ ]:
# ========================================
# 회귀계수 확인 (유의미한 변수 파악)
# ========================================

coef_df_ins = pd.DataFrame({
    'Variable': X_train_ins.columns,
    'Coefficient': lr_ins.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\n" + "="*60)
print("📊 문제 3-(2) 결과: 회귀계수")
print("="*60)
print(f"절편(Intercept): {lr_ins.intercept_:.4f}")
print(f"\n회귀계수 (절대값 기준 정렬):")
print(coef_df_ins.to_string(index=False))
print("\n💡 계수의 절대값이 클수록 영향력이 큼!")
print("="*60)

## 문제 3-(3) 회귀분석 성과 측정 (MAPE)

### 📌 요구사항
- 테스트집합으로 예측
- **MAPE로 성과 측정** ← ⭐ 핵심!

In [ ]:
# ========================================
# 3-(3) 예측 및 MAPE 측정
# ========================================

# 예측
y_pred_train_ins = lr_ins.predict(X_train_ins)
y_pred_test_ins = lr_ins.predict(X_test_ins)

# MAPE 계산 (⭐ 문제에서 요구!)
mape_train = mean_absolute_percentage_error(y_train_ins, y_pred_train_ins) * 100
mape_test = mean_absolute_percentage_error(y_test_ins, y_pred_test_ins) * 100

# 추가 지표들
rmse_test = np.sqrt(mean_squared_error(y_test_ins, y_pred_test_ins))
mae_test = mean_absolute_error(y_test_ins, y_pred_test_ins)
r2_test = lr_ins.score(X_test_ins, y_test_ins)

print("="*60)
print("📊 문제 3-(3) 결과: 회귀분석 성과")
print("="*60)
print(f"\n[학습 데이터]")
print(f"MAPE: {mape_train:.2f}%")

print(f"\n[테스트 데이터]  ← ⭐ 답안!")
print(f"MAPE: {mape_test:.2f}%  ← ⭐ 문제에서 요구한 지표!")
print(f"\n[추가 지표]")
print(f"RMSE: {rmse_test:,.2f}")
print(f"MAE:  {mae_test:,.2f}")
print(f"R²:   {r2_test:.4f}")
print("="*60)

## 문제 3-(4) Lasso 하이퍼파라미터 튜닝

### 📌 요구사항
- **학습집합만** 사용 (테스트집합 사용 X)
- alpha 파라미터 튜닝
- **10겹 교차검증**
- **MSE로 평가** ← ⭐ scoring='neg_mean_squared_error'

In [ ]:
# ========================================
# 3-(4) Lasso alpha 튜닝
# ========================================

print("="*60)
print("🔧 Lasso 하이퍼파라미터 튜닝 (alpha)")
print("="*60)

# alpha 후보 정의
param_grid_lasso = {
    'alpha': np.logspace(-4, 2, 100)  # 0.0001 ~ 100 사이 100개 값
}

# GridSearchCV 설정
grid_search_lasso = GridSearchCV(
    Lasso(max_iter=10000, random_state=42),
    param_grid_lasso,
    cv=10,                          # ← ⭐ 10겹 교차검증
    scoring='neg_mean_squared_error',  # ← ⭐ MSE로 평가
    n_jobs=-1,
    verbose=1
)

print("\n⏳ GridSearchCV 진행 중... (학습 데이터만 사용)")
grid_search_lasso.fit(X_train_ins, y_train_ins)  # ← 학습 데이터만!

# 최적 alpha
best_alpha_lasso = grid_search_lasso.best_params_['alpha']
best_mse_cv = -grid_search_lasso.best_score_  # neg를 원래대로

print("\n" + "="*60)
print("📊 문제 3-(4) 결과: 최적 alpha")
print("="*60)
print(f"최적 alpha: {best_alpha_lasso:.6f}  ← ⭐ 답안!")
print(f"10-Fold CV MSE: {best_mse_cv:,.2f}")
print("="*60)

## 문제 3-(5) Lasso 최종 성과 (MAPE)

### 📌 요구사항
- 3-(4)에서 찾은 최적 alpha 사용
- 학습집합으로 학습
- 테스트집합으로 예측
- **MAPE로 성과 측정** ← ⭐ MSE에서 MAPE로 변경!

In [ ]:
# ========================================
# 3-(5) 최적 Lasso 모델 성과 측정
# ========================================

# 최적 모델 가져오기
best_lasso = grid_search_lasso.best_estimator_

# 테스트 데이터 예측
y_pred_test_lasso = best_lasso.predict(X_test_ins)

# MAPE 계산 (⭐ 문제에서 요구!)
mape_test_lasso = mean_absolute_percentage_error(y_test_ins, y_pred_test_lasso) * 100

# 추가 지표
mse_test_lasso = mean_squared_error(y_test_ins, y_pred_test_lasso)
rmse_test_lasso = np.sqrt(mse_test_lasso)
mae_test_lasso = mean_absolute_error(y_test_ins, y_pred_test_lasso)

print("="*60)
print("📊 문제 3-(5) 결과: Lasso 최종 성과")
print("="*60)
print(f"\n최적 alpha: {best_alpha_lasso:.6f}")
print(f"\n[테스트 데이터 성과]  ← ⭐ 답안!")
print(f"MAPE: {mape_test_lasso:.2f}%  ← ⭐ 문제에서 요구한 지표!")
print(f"\n[추가 지표]")
print(f"MSE:  {mse_test_lasso:,.2f}")
print(f"RMSE: {rmse_test_lasso:,.2f}")
print(f"MAE:  {mae_test_lasso:,.2f}")
print("="*60)

In [ ]:
# ========================================
# Linear Regression vs Lasso 비교
# ========================================

print("\n" + "="*60)
print("🏆 모델 비교: Linear Regression vs Lasso")
print("="*60)

comparison_df_reg = pd.DataFrame({
    'Model': ['Linear Regression', 'Lasso'],
    'MAPE (%)': [mape_test, mape_test_lasso],
    'RMSE': [rmse_test, rmse_test_lasso],
    'MAE': [mae_test, mae_test_lasso]
})

print("\n", comparison_df_reg.to_string(index=False))

# 더 좋은 모델
if mape_test_lasso < mape_test:
    print(f"\n✅ Lasso가 더 우수 (MAPE 차이: {mape_test - mape_test_lasso:.2f}%p)")
else:
    print(f"\n✅ Linear Regression이 더 우수 (MAPE 차이: {mape_test_lasso - mape_test:.2f}%p)")
    
print("="*60)

---

# 🎯 최종 정리 및 답안 요약

## 문제 1. 로지스틱 회귀 (Phishing)
- **(3) 테스트 F1 Score**: 출력된 값 확인
- **(4) 5-Fold CV F1 Score 평균**: 출력된 값 확인
- **(5) 최적 C 값**: 출력된 값 확인
- **(5) 최고 CV ROC AUC**: 출력된 값 확인

## 문제 2. 의사결정트리 (Phishing)
- **(1) 최대 나무 F1 Score**: 출력된 값 확인
- **(2) 최적 max_depth**: 출력된 값 확인
- **(2) 최적 모델 F1 Score**: 출력된 값 확인
- **(3) 변수 중요도 Top 10**: 테이블 확인

## 문제 3. 회귀분석 (Insurance)
- **(2) 유의미한 변수**: 회귀계수 테이블의 절대값 큰 변수들
- **(3) Linear Regression MAPE**: 출력된 값 확인
- **(4) 최적 Lasso alpha**: 출력된 값 확인
- **(4) 10-Fold CV MSE**: 출력된 값 확인
- **(5) Lasso MAPE**: 출력된 값 확인

## 💡 시험 팁
1. **⭐ 표시된 부분**이 문제에서 요구한 핵심 답안
2. **평가 지표 주의**: F1 Score, ROC AUC, MAPE, MSE 등
3. **교차검증 횟수 확인**: 5겹 vs 10겹
4. **solver, max_depth 등 특정 파라미터** 요구사항 체크
5. 결과 출력 후 **테이블과 그래프**로 확인